# ML Feedback Loop Fairness Degradation Simulator

Razred simulira iterativno učenje modela, kjer se napovedi iz prejšnjih iteracij uporabljajo kot ciljne vrednosti v naslednjih fazah učenja, kar posnema povratne zanke (feedback loops) v realnih sistemih strojnega učenja. Nato prikaže spremembe metrik skozi čas iteracij.

### Potek simulacije

- **Iteracija 1:**  
  Del 1 → train → Del 2 → test (napovedi se shranijo in obravnavajo kot »resnične« vrednosti)

- **Iteracija 2:**  
  Del 2 → train (z napovedmi iz iteracije 1) → Del 3 → test

- **Naslednje iteracije:**  
  Vsaka iteracija uporablja napovedi prejšnje iteracije kot ciljno spremenljivko za učenje.

---

### Opazovane metrike

**Klasifikacija:**
- Accuracy, F1-score, Precision, Recall
- Samo pri **binarni**: Selection Rate, TPR (True Positive Rate), FNR (False Negative Rate), DPD (Demographic Parity Difference), EOD (Equalized Odds Difference)

**Regresija:**
- MAE, MSE, R², Mean residual (Povprečni ostanek), Standard residual (Standardni odklon ostankov)

# Nalaganje knjižnic

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from ipywidgets import IntSlider, interact
from IPython.display import display

from feedback_loop_simulator import FeedbackLoopFairnessDegradationSimulator


# Simulacija 1 - klasifikacija: binary

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from fairlearn.datasets import fetch_adult

data = fetch_adult()
df = data.frame
model = RandomForestClassifier(random_state=42)

sim = FeedbackLoopFairnessDegradationSimulator(
    model=model,
    dataset=df,
    target_variable="class",
    sensitive_variables=['sex', 'race'],
    display_func=display
)

# Deactivate iteration plotting (for higher iteration number)
sim.set_show_iteration_plots(False)

slider = IntSlider(
    value=3, 
    min=3, 
    max=100, 
    step=1, 
    description="Iterations", 
    continuous_update=False
)

_ = interact(sim.run_simulation, n_iterations=slider)

# Simulacija 2 - klasifikacija: multiclass (sintetična množica)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

np.random.seed(42)

n_samples = 6000

data = {
    'age': np.random.randint(18, 70, n_samples),
    'education_years': np.random.randint(8, 20, n_samples),
    'experience': np.random.randint(0, 40, n_samples),
    'skill_score': np.random.normal(50, 15, n_samples),
    'performance_rating': np.random.uniform(1, 10, n_samples),
}

# Sensitive variables
data['gender'] = np.random.choice(['Male', 'Female'], n_samples, p=[0.55, 0.45])
data['ethnicity'] = np.random.choice(['Group_A', 'Group_B', 'Group_C'], n_samples, p=[0.5, 0.3, 0.2])

df = pd.DataFrame(data)

# Create multiclass target (job_level: Junior, Mid, Senior, Lead) with BUILT-IN BIAS
base_score = (
    df['education_years'] * 3 +
    df['experience'] * 2 +
    df['skill_score'] * 0.5 +
    df['performance_rating'] * 5 +
    df['age'] * 0.2
)

# Add systematic bias based on sensitive variables
gender_bias = np.where(df['gender'] == 'Female', -15, 0)  # Women systematically disadvantaged
ethnicity_bias = df['ethnicity'].map({'Group_A': 0, 'Group_B': -10, 'Group_C': -20})  # Ethnic discrimination

# Final score with noise
final_score = base_score + gender_bias + ethnicity_bias + np.random.normal(0, 10, n_samples)

# Convert score to multiclass target (4 classes)
df['job_level'] = pd.cut(
    final_score,
    bins=[-np.inf, 80, 110, 140, np.inf],
    labels=['Junior', 'Mid', 'Senior', 'Lead']
)

model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    max_depth=8
)

sim = FeedbackLoopFairnessDegradationSimulator(
    model=model,
    dataset=df,
    target_variable='job_level',
    sensitive_variables=['gender', 'ethnicity'],
    display_func=display
)

slider = IntSlider(
    value=3, 
    min=3, 
    max=10, 
    step=1, 
    description="Iterations", 
    continuous_update=False
)

_ = interact(sim.run_simulation, n_iterations=slider)

# Simulacija 3 - regresija

In [ ]:
from fairlearn.datasets import fetch_boston
from sklearn.ensemble import RandomForestRegressor

boston = fetch_boston(as_frame=True, warn=False)
df = boston.frame

# Categorize B column (race proxy)
df['race_proxy'] = pd.cut(
    df['B'],
    bins=[0, 200, 400, 1000],
    labels=['High_minority', 'Medium_minority', 'Low_minority']
)

model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=8)

sim = FeedbackLoopFairnessDegradationSimulator(
    model=model,
    dataset=df,
    target_variable='MEDV',
    sensitive_variables=['race_proxy'],
    display_func=display
)

slider = IntSlider(
    value=3, 
    min=3, 
    max=10, 
    step=1, 
    description="Iterations", 
    continuous_update=False
)

_ = interact(sim.run_simulation, n_iterations=slider)

# Simulacija 4 - regresija (sintetična množica)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)

n_samples = 5000

data = {
    'feature_1': np.random.normal(50, 15, n_samples),
    'feature_2': np.random.normal(100, 25, n_samples),
    'feature_3': np.random.uniform(0, 10, n_samples),
    'feature_4': np.random.uniform(20, 80, n_samples),
}

# sensitive variables
data['gender'] = np.random.choice(['Male', 'Female'], n_samples, p=[0.6, 0.4])
data['region'] = np.random.choice(['North', 'South', 'East'], n_samples, p=[0.4, 0.35, 0.25])

df = pd.DataFrame(data)

# Create continuous target (income) with BUILT-IN BIAS
base_target = (
    df['feature_1'] * 2 +
    df['feature_2'] * 1.5 +
    df['feature_3'] * 10 +
    df['feature_4'] * 0.5
)

# Add systematic bias based on sensitive variables
gender_bias = np.where(df['gender'] == 'Female', -50, 0)  # Women systematically undervalued
region_bias = df['region'].map({'North': 0, 'South': -30, 'East': -60})  # Regional discrimination

# Final target with noise
df['income'] = base_target + gender_bias + region_bias + np.random.normal(0, 20, n_samples)


model = RandomForestRegressor(
    n_estimators=50,
    random_state=42,
    max_depth=6
)

sim = FeedbackLoopFairnessDegradationSimulator(
    model=model,
    dataset=df,
    target_variable='income',
    sensitive_variables=['gender', 'region'],
    display_func=display
)

slider = IntSlider(
    value=3, 
    min=3, 
    max=10, 
    step=1, 
    description="Iterations", 
    continuous_update=False
)

_ = interact(sim.run_simulation, n_iterations=slider)

# TODO:

* For higher number of iterations, option to turn off iteration plots
* ...



